In [1]:
import owlready2 as owl
import types
import json
from pprint import pprint as pp

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
ipro = owl.get_ontology("file://ipro.owl").load()
print(list(ipro.classes()))

[ipro.Album, ipro.Band, ipro.Bass, ipro.MusicalInstrument, ipro.Bassist, ipro.Musician, ipro.Discography, ipro.Drummer, ipro.Genre, ipro.Guitar, ipro.Guitarist, ipro.Keyboard, ipro.Keyboardist, ipro.Organ, ipro.Participation, ipro.Percussions, ipro.Piano, ipro.Singer, ipro.Song, ipro.Synthesizer, ipro.Vocals]


In [3]:
with open('wikidata/artists.json', 'r') as f:
    wikidata_artists = json.load(f)

with open('discogs/artists.json', 'r') as f:
    tmp = json.load(f)
    discogs_artists = {e['discogsID']: e['artist'] for e in tmp}

with ipro:
    class wikidata_id(ipro.Band >> str, owl.FunctionalProperty): pass
    class discogs_id(ipro.Band >> str, owl.FunctionalProperty): pass
    class progarchives_id(ipro.Band >> str, owl.FunctionalProperty): pass

    for artist in wikidata_artists:
        band = ipro.Band(artist['itemLabel']['value'].replace(' ', '_').replace('\'', ''))
        band.wikidata_id = artist['item']['value'].split('/')[-1]
        band.discogs_id = artist['discogsId']['value']
        band.progarchives_id = artist['progarchivesId']['value']
        band.comment = [owl.locstr(discogs_artists[band.discogs_id]['profile'], lang = 'en')]
        band.label = [owl.locstr(discogs_artists[band.discogs_id]['name'], lang = 'en')]

pp(list(ipro.Band.instances())[:10])

[ipro.Acqua_Fragile,
 ipro.Alphataurus,
 ipro.Analogy,
 ipro.Area,
 ipro.Arti_e_Mestieri,
 ipro.Banco_del_Mutuo_Soccorso,
 ipro.Blocco_Mentale,
 ipro.Cervello,
 ipro.De_De_Lind,
 ipro.Formula_3]


In [4]:
# print all info of Acqua Fragile
pp(ipro.Acqua_Fragile)
pp(ipro.Acqua_Fragile.comment)
pp(ipro.Acqua_Fragile.comment[0].lang)
pp(ipro.Acqua_Fragile.label)
pp(ipro.Acqua_Fragile.label[0].lang)
pp(ipro.Acqua_Fragile.wikidata_id)
pp(ipro.Acqua_Fragile.discogs_id)
pp(ipro.Acqua_Fragile.progarchives_id)

ipro.Acqua_Fragile
['Acqua Fragile was a progressive rock band from Parma, Italy. They were '
 'formed in 1971 by three former members of Gli Immortali: singer and '
 'guitarist [a=Bernardo Lanzetti], guitarist [a=Gino Campanini], and drummer '
 '[a=Piero Canavera]. They were joined by former [a3640528] bassist [a=Franz '
 'Dondi] and keyboardist [a=Maurizio Mori]. After being spotted by the very '
 'successful [a=Premiata Forneria Marconi] (PFM) they were signed to Numero '
 'Uno. This Line-Up recorded two albums, a self titled debut (1973) and Mass '
 'Media Stars (1974). Maurizio Mori left after the second album and was '
 'replaced by [a=Joe Vescovi]. The Line-Up with Joe Vescovi appears on the '
 'posthumously released Live In Emilia: Spring 75 (1994). Bernardo Lanzetti '
 'left to join PFM in 1975 and was briefly replaced by [a=Roby Facini]. By the '
 'end of the year the group had broken up. Since 2005 Franz Dondi played '
 'material Live under the name Acqua Fragile Project. Re